In [ ]:
from astroquery.gaia import Gaia
import warnings

# Comment this out if you want to see warnings
warnings.filterwarnings('ignore')
import astropy
import requests
import pyvo as vo
import numpy as np
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import glob
import os
import requests
import sys
sys.path.append("../gaia_tools/")
import data_analysis
import covariance_generation as cov
from import_functions import import_data

In [ ]:
# Original sample
gaia_path = "/hdfs/local/sven/gaia_tools_data/gaia_rv_data_bayes.csv"
icrs_data = import_data(path = gaia_path, is_bayes = True, debug = False)

In [ ]:
tmass_data_path = '/hdfs/local/sven/gaia_tools_data/crossmatched_tmass_data/crossmatched_tmass_data.csv'
crossmatched_tmass_data = pd.read_csv(tmass_data_path)

In [ ]:
merged_data = icrs_data.merge(crossmatched_tmass_data, on='source_id', suffixes=('_gaia', '_tmass'))

In [ ]:
print(crossmatched_tmass_data.columns)
print(crossmatched_tmass_data.shape)

### Color range cut

In [ ]:
crossmatched_tmass_data[((crossmatched_tmass_data.j_m - crossmatched_tmass_data.k_m) > 0.5) & ((crossmatched_tmass_data.j_m - crossmatched_tmass_data.k_m) < 1.1)]

### Uncertainty cut

In [ ]:
crossmatched_tmass_data[(crossmatched_tmass_data.j_msigcom) < 0.1]
crossmatched_tmass_data[(crossmatched_tmass_data.h_msigcom) < 0.1]
crossmatched_tmass_data[(crossmatched_tmass_data.k_msigcom) < 0.1]

### Photometric quality flag cut

In [ ]:
import re
def is_allowed_flg(string):

    charRe = re.compile(r'[^A-B]')
    string = charRe.search(string)
    return not bool(string)

#crossmatched_tmass_data.apply(lambda row : is_allowed_flg(row.ph_qual), axis=1)


In [ ]:
sliced_tmass_data = crossmatched_tmass_data[((crossmatched_tmass_data.j_m - crossmatched_tmass_data.k_m) > 0.5) & ((crossmatched_tmass_data.j_m - crossmatched_tmass_data.k_m) < 1.1)]

sliced_tmass_data = sliced_tmass_data[((sliced_tmass_data.j_msigcom) < 0.1) & ((sliced_tmass_data.h_msigcom) < 0.1) & ((sliced_tmass_data.k_msigcom) < 0.1)]

sliced_tmass_data =  sliced_tmass_data[sliced_tmass_data.apply(lambda row : is_allowed_flg(row.ph_qual), axis=1) == True]

In [ ]:
sliced_tmass_data.reset_index(inplace=True, drop=True)
sliced_tmass_data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def scatter_hist(x, y, ax, ax_histx, ax_histy):
    # no labels
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)

    # the scatter plot:
    ax.scatter(x, y, alpha=0.1, s=0.5)
    ax.set_xlim(0.4, 1.4)
    ax.set_ylim(5, 14)

    ax.set_xlabel(r'$J - K_s$ [mag]', fontdict={'fontsize' : 14})
    ax.set_ylabel(r'$H$ [mag]',  fontdict={'fontsize' : 14})

    ax.vlines([0.5, 1.1], 5, 14, linestyles='dashed', alpha=0.5)

    # now determine nice limits by hand:
    binwidth = 0.25
    xymax = max(np.max(np.abs(x)), np.max(np.abs(y)))
    lim = (int(xymax/binwidth) + 1) * binwidth

    bins = np.arange(-lim, lim + binwidth, binwidth)
    bins = 20
    ax_histx.hist(x, bins=bins, histtype='step', linewidth = 2)
    ax_histy.hist(y, bins=bins, histtype='step', linewidth = 2, orientation='horizontal')

    ax_histx.set_yticks([])
    ax_histy.set_xticks([])


In [ ]:
x_values = sliced_tmass_data.j_m - sliced_tmass_data.k_m
y_values = sliced_tmass_data.h_m

# definitions for the axes
left, width = 0.1, 0.65
bottom, height = 0.1, 0.65
spacing = 0.005

rect_scatter = [left, bottom, width, height]
rect_histx = [left, bottom + height + spacing, width, 0.2]
rect_histy = [left + width + spacing, bottom, 0.2, height]

# start with a square Figure
fig = plt.figure(figsize=(8, 8))

ax = fig.add_axes(rect_scatter)
ax_histx = fig.add_axes(rect_histx, sharex=ax)
ax_histy = fig.add_axes(rect_histy, sharey=ax)

# use the previously defined function
scatter_hist(x_values[:: 10],y_values[:: 10], ax, ax_histx, ax_histy)

#plt.show()

fig_name = 'gaia_data_bovy_cuts'
#plt.savefig('/home/sven/repos/gaia-tools/out/crossmatch_sanity_checks/' + fig_name +'.png', dpi=300)

---